In [2]:
%pip install kagglehub

  Using cached cryptography-44.0.0-cp39-abi3-win_amd64.whl.metadata (5.7 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached cryptography-44.0.0-cp39-abi3-win_amd64.whl (3.2 MB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
   ---------------------------------------- 2.2/2.2 MB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 870.8/870.8 kB 2.3 MB/s eta 0:00:00
  Created wheel for grpclib: filename=grpclib-0.4.7-py3-none-any.whl size=76255 sha256=31753b47ae29877ad6fbbe955cb077c2b86a9768a31a68d5574b3a670e6e816e
  Stored in directory: c:\users\scxrp\appdata\local\pip\cache\wheels\05\c0\1c\3d8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 2.14.1 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
pyasn1-modules 0.3.0 requires pyasn1<0.6.0,>=0.4.6, but you have pyasn1 0.6.1 which is incompatible.
streamlit 1.28.2 requires pillow<11,>=7.1.0, but you have pillow 11.1.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
'''
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kushagra3204/wheat-plant-diseases")

print("Path to dataset files:", path)
'''

100%|██████████| 6.09G/6.09G [31:37<00:00, 3.45MB/s]  

Extracting files...


Path to dataset files: C:\Users\scxrp\.cache\kagglehub\datasets\kushagra3204\wheat-plant-diseases\versions\6


In [25]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import os

In [24]:
print("version", tf.version)

version <module 'tensorflow._api.v2.version' from 'c:\\Users\\scxrp\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\tensorflow\\_api\\v2\\version\\__init__.py'>


In [37]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 0


In [26]:
#seeds
tf.random.set_seed(42)
np.random.seed(42)

In [27]:
#definitions
IMG_SIZE = 300
BATCH_SIZE = 32
NUM_CLASSES = 15
EPOCHS = 50

### Checking all the data is proper

In [28]:
data_dir = 'D:\code_stuff\codmav\wheatdiseases1\data'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

In [29]:
train_classes = sorted(os.listdir(train_dir))
val_classes = sorted(os.listdir(val_dir))
test_classes = sorted(os.listdir(test_dir))

In [30]:
assert train_classes == val_classes == test_classes, "Class folders don't match across train/val/test"
print("Found classes:", train_classes)
print(f"Number of classes: {len(train_classes)}")

Found classes: ['aphid', 'black_rust', 'blast', 'brown_rust', 'common_root_rot', 'fusarium_head_blight', 'healthy', 'leaf_blight', 'mildew', 'mite', 'septoria', 'smut', 'stem_fly', 'tan_spot', 'yellow_rust']
Number of classes: 15


In [31]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

def create_generator(directory, is_training=False):
    return datagen.flow_from_directory(
        directory,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='sparse',
        shuffle=is_training
    )

train_generator = create_generator(train_dir, is_training=True)
val_generator = create_generator(val_dir)
test_generator = create_generator(test_dir)

Found 13104 images belonging to 15 classes.
Found 300 images belonging to 15 classes.
Found 750 images belonging to 15 classes.


In [32]:
base_model = tf.keras.applications.EfficientNetB3(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

In [33]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [34]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [35]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

In [36]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

Epoch 1/50
 12/410 [..............................] - ETA: 15:40 - loss: 4.1843 - accuracy: 0.0755

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")

In [ ]:
predictions = model.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(15, 15))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=train_classes,
            yticklabels=train_classes)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=train_classes))

In [ ]:
model.save('wheat_effnetb3.h5')